In [3]:
from data.data import Delpher, DBNL
from nltk.corpus import stopwords
import spacy
import os
import nltk
import re
import string
import gensim
import numpy as np

# for tokenization
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('wordnet')

# for stopword removal
from nltk.corpus import stopwords
nltk.download('stopwords')

# for lemmatization and POS tagging
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

# for LDA
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# for LDA evaluation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvisualize
nlp = spacy.load('xx_sent_ud_sm', disable=['parser', 'ner'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\imruh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\imruh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\imruh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\imruh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
years = [1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819]

In [5]:
delpher = Delpher().dataset.filter(lambda example: example["Year"] in years)
dbnl = DBNL().dataset.filter(lambda example: example["Year"] in years)

In [6]:
delpher_text = delpher["CleanedText"]
dbnl_text = dbnl["CleanedText"]

In [7]:
def preprocess_text(text):
    text = re.sub('\s+', ' ', text)  # Remove extra spaces
    text = re.sub('\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub('\'', '', text)  # Remove apostrophes
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    text = text.lower()  # Convert to lowercase
    return text

<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:3: DeprecationWarning: invalid escape sequence '\S'
C:\Users\imruh\AppData\Local\Temp\ipykernel_33396\3118430235.py:2: DeprecationWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)  # Remove extra spaces
C:\Users\imruh\AppData\Local\Temp\ipykernel_33396\3118430235.py:3: DeprecationWarning: invalid escape sequence '\S'
  text = re.sub('\S*@\S*\s?', '', text)  # Remove emails


In [8]:
delpher_text = [preprocess_text(x) for x in delpher_text]
dbnl_text = [preprocess_text(x) for x in dbnl_text]

In [9]:
dataset_cleaned_text = delpher_text + dbnl_text

In [10]:
stoplist = list(set(stopwords.words('dutch'))) + list(set(stopwords.words('english'))) + list(set(stopwords.words('french')))
stoplist.extend(['pct', 'idem', 'wy', 'zyn' ,'zy','hy', 'pcts', 'dito', 'zoo', 'alhier'])

# Tokenize and remove stopwords
def tokenize(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    tokens = [token for token in tokens if token not in stoplist]
    return tokens

tokens = [tokenize(x) for x in dataset_cleaned_text]

In [11]:
def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

lemmas = [lemmatize(x) for x in tokens]

In [12]:
id2word = corpora.Dictionary(lemmas)
texts = lemmas
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
# train LDA model
lda_model = LdaModel(corpus=corpus, 
                     id2word=id2word, 
                     random_state=4583, 
                     chunksize=20, 
                     num_topics=10, 
                     passes=200, 
                     iterations=400)

KeyboardInterrupt: 

In [ ]:
# print LDA topics
for topic in lda_model.print_topics(num_topics=7, num_words=10):
    print(topic)